# Caption Generation from Images CNN & RNN キャプション生成

![title](https://cdn-images-1.medium.com/max/1200/0*mCHDMNdwb_gB1Rj9.)

### Task
与えられた画像の説明文（キャプション）の作成を自動化するAI
### Architecture
CNN & LSTM  転移学習
### Framework
Keras ( TensorFlow backend )
### Reserch Paper
Where to put the Image in an Image Caption Generator (https://arxiv.org/abs/1703.09137)  
Oxford Visual Geometry Group, or VGG, model that won the ImageNet competition in 2014   
Very Deep Convolutional Networks for Large-Scale Visual Recognition (http://www.robots.ox.ac.uk/~vgg/research/very_deep/)
### Process
0. タスク定義
1. データ準備（Flickr 8k）
2. ニューラルネット構築（転移学習）
3. 学習&テスト
4. 評価&改善（BLEU scores）

## 1. Data Preparation

- 写真の内容を解釈するために、事前に訓練されたモデルを使用します。
- その後、これらの機能を後でロードして、データセット内の特定の写真の解釈としてモデルに入力できます。  
- VGGクラスを使用して、KerasにVGGモデルをロードできます。
- これは写真の分類を予測するために使用されるモデルなので、ロードされたモデルから最後のレイヤーを削除します。
- 画像の分類には興味がありませんが、分類が行われる直前の写真の内部表現には興味があります。
- これらは、モデルが写真から抽出した「特徴」です。
- Kerasはまた、ロードした写真をモデルに適したサイズに変形するためのツールも提供します（例：3チャンネル224 x 224ピクセルの画像）。
- 下はextract_features（）という名前の関数です。
- ディレクトリ名を指定すると、各写真をロードしてVGG用に準備し、VGGモデルから予測された特徴を収集します。

In [22]:
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint

In [ ]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

### A) Prepare Text Data 画像から特徴量を生成

#### VGG16 is a CNN architecture named Visual Geometry Group from Oxford


##### 大量の画像データに対して、VGG16を用いた転移学習を実行する

- 転移学習：既存の学習済モデル（出力層以外の部分）を、重みデータは変更せずに特徴量抽出機として利用する
- ファインチューニング：既存の学習済モデル（出力層以外の部分）を、重みデータを一部再学習して特徴量抽出機として利用する
  
###### 転移学習の手順
1. 入力画像から、特徴量(ボトルネック特徴量)を抽出する
2. ボトルネック特徴量を用いて、クラス分類をする
  
　つまり転移学習では、VGG16など大規模なデータを用いて学習した強力なモデルを特徴抽出器として利用し（多数の対象を分類できる為、画像の特徴を捉えるのが非常に上手い）、任意のクラスの分類する為の特徴量の圧縮器として利用しています。

In [ ]:
def extract_features(curr_dir):
    # CNNの一つであるVGG16モデルで転移学習を行う
    model = VGG16() 
    # モデルの再構築
    model.layers.pop()
    model = Model( inputs = model.inputs, outputs = model.layers[-1].output )
    # モデルの統合
    print(model.summary())
    
    features = dict()
    # 各画像から特徴を生成する
    for name in listdir(curr_dir):
        filename = curr_dir + "/" + name
        image = load_img( filename, target_size=(224, 224) )
        # 計算の高速化のために画像のピクセル数をndarrayに変換する
        image = img_to_array(image)
        # 画像データを4次元アレイに変換する
        image = image.reshape(( 1, image.shape[0], image.shape[1], image.shape[2] ))
        # VGG16モデルへ入力するため、画像データを最終処理
        image = preprocess_input(image)
        # 特徴量を生成 ( verbose: 0, 1または2．詳細表示モード．0とすると標準出力にログを出力しません )
        feature = model.predict( image, verbose = 0 )
        # 画像のIDを生成
        image_id = name.split(".")[0]
        features[image_id] = feature
        print( ">%s"%name )
    
    # 戻り値はkey=画像ID, value=画像の特徴量という辞書 
    return features

上記の関数にデータを流し込んで特徴量生成を実行する

In [ ]:
curr_dir = "/Users/akr712/Desktop/CaptionGeneration_CNNandLSTM_Keras/Flicker8k_Dataset"
features = extract_features(curr_dir)
print( "生成された特徴量の次元数: %d" % len(features) )
# Pickleモジュールはプログラムを実行し終えたあとも作成したオブジェクトを保存する機能を提供してくれる
dump( features, open("features.pkl", "wb") )

### B) Prepare Text Data テキストから特徴量を生成

In [ ]:
def load_text(filename):
    """
    function: ファイル内のテキストを抽出する
    input: テキストファイル
    output: テキスト
    """
    file = open(filename, "r")
    text = file.read()
    file.close()
    return text

filename = "Flickr8k_text/Flickr8k.token.txt"
texts = load_text(filename)

## A CNN LSTM Model

#### Text Data の中身はこうなっています
  

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .  
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .  
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .  
1000268201_693b08cb0e.jpg#3	A little girl climbing the stairs to her playhouse .  
1000268201_693b08cb0e.jpg#4	A little girl in a pink dress going into a wooden cabin .  
1001773457_577c3a7d70.jpg#0	A black dog and a spotted dog are fighting  
1001773457_577c3a7d70.jpg#1	A black dog and a tri-colored dog playing with each other on the road .  
1001773457_577c3a7d70.jpg#2	A black dog and a white dog with brown spots are staring at each other in the street .  
1001773457_577c3a7d70.jpg#3	Two dogs of different breeds looking at each other on the road .  
1001773457_577c3a7d70.jpg#4	Two dogs on pavement moving toward each other .  

In [ ]:
def extract_descriptions(text):
    """
    
    
    
    """
    mapping = dict()
    
    for line in text.split("\n"):
        tokens = line.split()
        # descriptionがないものは除外
        if len(line) < 2:
            continue
        image_id, image_desc = tokens[0], [1:]
        image_id = image_id.split(".")[0]
        image_desc = " ".join(image_desc)
        if image_id not in mapping:
            mapping[image_id] = list()
        mapping[image_id].append(image_desc)
    
    return mapping

descriptions = extract_descriptions(texts)
print("ロード中: %d" % len(descriptions) )

In [ ]:
import string

def clean_descriptions(descriptions):
    """
    
    
    
    """
    table = str.maketrans(", ". string.punctuation)
    for key, desc_list in descriptions.items():
        
        for i in range(len(desc_list)):
            desc = desc_list[i]
            desc = desc.split()
            desc = [ word.lower(), for word in desc ]
            desc = [ punc.translate(table) for punc in desc ]
            desc = [ word for word in desc len(word) > 1 ]
            desc = [ word for word in desc if word.isalpha() ]
            desc_list[i] = "".join(desc)
            
clean_descriptions(descriptions)

In [ ]:
def desc2vocab(descriptions):
    all_desc = set()
    for key in descriptions.keys():
        for d in descriptions[key]:
            all_desc.update(d.split())
    return all_desc

vocabulary = desc2vocab(descriptions)
print("Vocabulary Size: %d" % len(vocabulary))

最後に、画像IDとDiscriptionを含む辞書をdescriptions.txtという新しいファイルに保存する

In [ ]:
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append( key + " " + desc )
    data = "\n".join(lines)
    file = open(filename, "w")
    file.write(data)
    file.close()
    
save_descriptions(descriptions, descriptions.txt)

## 2. Develop CNN-LSTM Model ニューラルネットのアーキテクチャを作成

![title](https://raw.githubusercontent.com/yunjey/pytorch-tutorial/master/tutorials/03-advanced/image_captioning/png/model.png)

In [10]:
def load_text(filename):
    """
    function: ファイル内のテキストを抽出する
    input: テキストファイル
    output: テキスト
    """
    file = open(filename, "r")
    text = file.read()
    file.close()
    return text

def load_data(filename):
    text = load_text(filename)
    data = list()
    
    for line in text.split("\n"):
        if len(line) < 1:
            continue
        identifier = line.split(".")[0]
        data.append(identifier)
    
    return set(data)

In [11]:
def load_clean_description(filename, data):
    text = load_text(filename)
    descriptions = dict()
    
    for line in text.split("\n"):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1:]
        
        if image_id not in descriptions:
            descriptions[image_id] = list()
        desc =  'startseq ' + ' '.join(image_desc) + ' endseq'
        descriptions[image_id].append(desc)
    return descriptions

画像から特徴量を抽出する

In [14]:
from pickle import load

def extract_image_features(filename, data):
    all_features = load(open(filename, "rb"))
    features = { k: all_features[k] for k in  data }
    return features

画像とでディスクリプションを出力する

In [17]:
filename = 'Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_data(filename)
print('Dataset: %d' % len(train))

# 画像の特徴量
train_features = load_photo_features('features.pkl', train)
print('Images: train=%d' % len(train_features))

# ディスクリプション
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

FileNotFoundError: [Errno 2] No such file or directory: 'Flickr8k_text/Flickr_8k.trainImages.txt'

In [18]:
def desc2line(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [ all_desc.append(d) for d in descriptions[key] ]
    return all_desc

In [19]:
def desc2token(descriptions):
    lines = desc2line(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

tokenizer = desc2token(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print("単語数 : %d" % vocab_size)

NameError: name 'train_descriptions' is not defined

画像についての文章を作成して、キャプションを生成する

In [20]:
def create_sequence(tokenizer, max_length, descriptions, images):
    """
    
    
    """
    X1, X2, y = list(), list(), list()
    # それぞれの画像をループ
    for key, desc_list in descriptions.items():
        # 画像のデスクリプションをループ
        for desc in desc_list:
            # デスクリプションをエンコード
            seq = tokenizer.texts_to_sequences([desc])[0]
            for i in range(1, len(seq)):
                # 入力文と出力文をペアにする
                in_seq, out_seq =seq[:i], seq[i]
                # 文を固定長に合わせる
                in_seq = pad_sequences( [in_seq], maxlen = max_length )[0]
                # 出力文をエンコードする
                out_seq = to_categorical( [out_seq], num_classes=vocab_size )[0]
                # 保存する
                X1.append(images[key][0])
                X2.append(in_seq)
                y.append(out_seq)
    
    return array(X1), array(X2), array(y)

最も単語数の多いデスクリプションの単語数をカウントする

In [21]:
def max_length(desceiptions):
    lines = desc2line(descriptions)
    return max(len(d.split()) for d in lines)

## Defining the Model :  “merge-model” described by Marc Tanti
- Where to put the Image in an Image Caption Generator, 2017.
- What is the Role of Recurrent Neural Networks (RNNs) in an Image Caption Generator?, 2017. https://arxiv.org/abs/1708.02043
   
![title](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/09/Schematic-of-the-Merge-Model-For-Image-Captioning.png)

![title](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/09/Plot-of-the-Caption-Generation-Deep-Learning-Model.png)

- Photo Feature Extractor. This is a 16-layer VGG model pre-trained on the ImageNet dataset. We have pre-processed the photos with the VGG model (without the output layer) and will use the extracted features predicted by this model as input.
- Sequence Processor. This is a word embedding layer for handling the text input, followed by a Long Short-Term Memory (LSTM) recurrent neural network layer.
- Decoder (for lack of a better name). Both the feature extractor and sequence processor output a fixed-length vector. These are merged together and processed by a Dense layer to make a final prediction.

キャプションを生成する関数を定義する

In [23]:
def caption_generator(vocab_size, max_length):
    """
    
    
    
    """
    # 特徴量を生成
    input1 = Input(shape = (4096,))
    feature1 = Dropout(0.5)(input1)
    feature2 = Dense(256, activation="relu")(feature1)
    
    # LSTMに入力するシーケンスに変換する
    input2 = Input(shape = (max_length,))
    seq1 = Embedding(vocab_size, 256, mask_zero=True)(input2)
    seq2 = Dropout(0.5)(seq1)
    seq3 = LSTM(256)(seq2)
    
    # decoderを生成
    decoder1 = add([feature2, seq3])
    decoder2 = Dense(256, activation="relu")(decoder1)
    output = Dense(vocab_size, activation='softmax')(decoder2)
    
    # [画像、ディスクリプション]と[単語]を結びつける
    model = Model( inputs=[input1, input2], outputs=output )
    # LSTMの出力層の活性化関数はAdam
    model.compile( loss = "'categorical_crossentropy", optimizer = "adam" )
    print(model.summary())
    plot_model(model, to_file="model.png", show_shapes=True)
    return model

In [25]:
# チェックポイントを作る
filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [29]:
# 学習スタート
model.fit( [X1train, X2train], ytrain, epochs = 20, verbose = 2, callbacks = [checkpoint], validation_data = ([X1test, X2test], ytest) )

NameError: name 'model' is not defined